In [12]:
import pandas as pd
from pymongo import MongoClient
import numpy as np

client = MongoClient('mongodb://<username>:<password>@ds211774.mlab.com:11774/<dbname>')
db=client.<dbaname>



Database(MongoClient(host=['ds211774.mlab.com:11774'], document_class=dict, tz_aware=False, connect=True), 'heroku_d920w02q')

In [ ]:
##Read in csv

In [13]:
import os
os.chdir("C:/dev/flask/data")

In [14]:
data=pd.read_csv("adult.csv")

In [15]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [18]:
##Assigning id for each record
data = data.assign(pid=np.arange(len(data))).reset_index(drop=True)


In [19]:
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,pid
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,1
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,2
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,3
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,4


In [20]:
##insert the columns needed to the "in house" database
datatomongo=data[['age','capital-gain','capital-loss','gender','native-country',
 'race','marital-status','education','educational-num','pid']]


In [21]:
col=db.income
col.insert_many(datatomongo.to_dict('records'))


In [166]:
##create replace dicts for use in prediction- ie transform categorical variables to cont

catnames=['workclass','education','marital-status','occupation','relationship','race','gender','native-country','income']
transformdict={}
for name in catnames:
    labels = data[name].astype('category').cat.categories.tolist()
    transformdict[name] = {k: v for k,v in zip(labels,list(range(0,len(labels))))}

transformdict


{'workclass': {'?': 0,
  'Federal-gov': 1,
  'Local-gov': 2,
  'Never-worked': 3,
  'Private': 4,
  'Self-emp-inc': 5,
  'Self-emp-not-inc': 6,
  'State-gov': 7,
  'Without-pay': 8},
 'education': {'10th': 0,
  '11th': 1,
  '12th': 2,
  '1st-4th': 3,
  '5th-6th': 4,
  '7th-8th': 5,
  '9th': 6,
  'Assoc-acdm': 7,
  'Assoc-voc': 8,
  'Bachelors': 9,
  'Doctorate': 10,
  'HS-grad': 11,
  'Masters': 12,
  'Preschool': 13,
  'Prof-school': 14,
  'Some-college': 15},
 'marital-status': {'Divorced': 0,
  'Married-AF-spouse': 1,
  'Married-civ-spouse': 2,
  'Married-spouse-absent': 3,
  'Never-married': 4,
  'Separated': 5,
  'Widowed': 6},
 'occupation': {'?': 0,
  'Adm-clerical': 1,
  'Armed-Forces': 2,
  'Craft-repair': 3,
  'Exec-managerial': 4,
  'Farming-fishing': 5,
  'Handlers-cleaners': 6,
  'Machine-op-inspct': 7,
  'Other-service': 8,
  'Priv-house-serv': 9,
  'Prof-specialty': 10,
  'Protective-serv': 11,
  'Sales': 12,
  'Tech-support': 13,
  'Transport-moving': 14},
 'relationshi

In [169]:
##transformdictreversed: this is for translating the numeric to categorical for readability in response
transformdictreversed = defaultdict(dict)
for key, val in transformdict.items():
    for subkey, subval in val.items():
        transformdictreversed[key][subval] = subkey
transformdictreversed=dict(transformdictreversed)
transformdictreversed

{'workclass': {0: '?',
  1: 'Federal-gov',
  2: 'Local-gov',
  3: 'Never-worked',
  4: 'Private',
  5: 'Self-emp-inc',
  6: 'Self-emp-not-inc',
  7: 'State-gov',
  8: 'Without-pay'},
 'education': {0: '10th',
  1: '11th',
  2: '12th',
  3: '1st-4th',
  4: '5th-6th',
  5: '7th-8th',
  6: '9th',
  7: 'Assoc-acdm',
  8: 'Assoc-voc',
  9: 'Bachelors',
  10: 'Doctorate',
  11: 'HS-grad',
  12: 'Masters',
  13: 'Preschool',
  14: 'Prof-school',
  15: 'Some-college'},
 'marital-status': {0: 'Divorced',
  1: 'Married-AF-spouse',
  2: 'Married-civ-spouse',
  3: 'Married-spouse-absent',
  4: 'Never-married',
  5: 'Separated',
  6: 'Widowed'},
 'occupation': {0: '?',
  1: 'Adm-clerical',
  2: 'Armed-Forces',
  3: 'Craft-repair',
  4: 'Exec-managerial',
  5: 'Farming-fishing',
  6: 'Handlers-cleaners',
  7: 'Machine-op-inspct',
  8: 'Other-service',
  9: 'Priv-house-serv',
  10: 'Prof-specialty',
  11: 'Protective-serv',
  12: 'Sales',
  13: 'Tech-support',
  14: 'Transport-moving'},
 'relationshi

In [88]:

#making data numeric
for name in catnames:
    data.replace(d[name], inplace=True)

data.head()



,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,rec_id
0,25,4,226802,1,7,4,7,3,2,1,0,0,40,39,0,0
1,38,4,89814,11,9,2,5,0,4,1,0,0,50,39,0,1
2,28,2,336951,7,12,2,11,0,4,1,0,0,40,39,1,2
3,44,4,160323,15,10,2,7,0,2,1,7688,0,40,39,1,3
4,18,0,103497,15,10,4,0,3,4,0,0,0,30,39,0,4


In [91]:
##defining X,Y
Y=data['income']
X=data[['age','workclass','education','educational-num','marital-status','occupation','relationship','race','gender',
        'capital-gain','capital-loss','hours-per-week','native-country']]


In [93]:
X.head()

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country
0,25,4,1,7,4,7,3,2,1,0,0,40,39
1,38,4,11,9,2,5,0,4,1,0,0,50,39
2,28,2,7,12,2,11,0,4,1,0,0,40,39
3,44,4,15,10,2,7,0,2,1,7688,0,40,39
4,18,0,15,10,4,0,3,4,0,0,0,30,39


In [133]:
#Divide in test/train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [95]:
X_train.shape

(39073, 13)

In [101]:
#train model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
classifier.fit(X_train, y_train)

#make predictions
y_pred = classifier.predict(X_test)


In [100]:
y_pred = classifier.predict(X_test)


In [104]:
#evaluate model
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_pred)
print(accuracy)

0.8508547446002661


In [106]:
#export model
from joblib import dump, load
dump(classifier, 'model.joblib')

model = load('model.joblib') 


In [128]:
import os
print(os.getcwd())

C:\dev\flask\data


In [124]:
#predictions
ymodel=model.predict(X_test)
#predictions probabilites
yprobs=model.predict_proba(X_test)[:,1]

In [110]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,ymodel)
print(accuracy)

0.8508547446002661


In [136]:

d={'age': 20,
 'workclass': 4,
 'education': 11,
 'educational-num': 9,
 'marital-status': 2,
 'occupation': 5,
 'relationship': 0,
 'race': 4,
 'gender': 1,
 'capital-gain': 0,
 'capital-loss': 0,
 'hours-per-week': 50,
 'native-country': 39}
